<a href="https://colab.research.google.com/github/eric15342335/realfill/blob/main/train_realfill.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/eric15342335/realfill
%cd realfill

In [ ]:
!curl -L https://github.com/eric15342335/realfill/releases/download/dataset/realfill_data_release_full.zip -o realfill_data_release_full.zip
!unzip -q realfill_data_release_full.zip

In [ ]:
%pwd
%ls -lh

In [ ]:
!uv pip install -U -r requirements.txt --no-progress

In [ ]:
from accelerate.utils import write_basic_config
write_basic_config()

In [ ]:
import os

# Toggle switch - set to True for Google Drive storage, False for local storage
USE_DRIVE_STORAGE = True

# Base directory for Google Drive storage
DRIVE_BASE_DIR = "/content/drive/MyDrive/RealFill"

# Keep your original environment variable setup
os.environ["DATASET"] = "realfill_data_release_full"
os.environ["MODEL_NAME"] = "stabilityai/stable-diffusion-2-inpainting"
os.environ["BENCHMARK"] = "RealBench"  # Replace with "Qualitative" if needed
os.environ["DATASET_NUMBER"] = "25"

# Construct paths based on the storage toggle
if USE_DRIVE_STORAGE:
    # Use Google Drive for storage
    from google.colab import drive
    drive.mount('/content/drive')
    os.environ["OUTPUT_DIR"] = f'{DRIVE_BASE_DIR}/{os.environ["BENCHMARK"]}-{os.environ["DATASET_NUMBER"]}-model'
    os.environ["OUTPUT_IMG_DIR"] = f'{DRIVE_BASE_DIR}/{os.environ["BENCHMARK"]}-{os.environ["DATASET_NUMBER"]}-results'
else:
    # Use local Colab storage (original paths)
    os.environ["OUTPUT_DIR"] = f'{os.environ["BENCHMARK"]}-{os.environ["DATASET_NUMBER"]}-model'
    os.environ["OUTPUT_IMG_DIR"] = f'{os.environ["BENCHMARK"]}-{os.environ["DATASET_NUMBER"]}-results'

# The remaining variable paths stay the same
os.environ["TRAIN_DIR"] = f'{os.environ["DATASET"]}/{os.environ["BENCHMARK"]}/{os.environ["DATASET_NUMBER"]}'
os.environ["VAL_IMG"] = f'{os.environ["TRAIN_DIR"]}/target/target.png'
os.environ["VAL_MASK"] = f'{os.environ["TRAIN_DIR"]}/target/mask.png'

# Create the necessary directories
os.makedirs(os.environ["OUTPUT_DIR"], exist_ok=True)
os.makedirs(os.environ["OUTPUT_IMG_DIR"], exist_ok=True)

print(f"Using {'Google Drive' if USE_DRIVE_STORAGE else 'local Colab'} storage")
print(f"Output directory: {os.environ['OUTPUT_DIR']}")

In [ ]:
!accelerate launch train_realfill.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --train_data_dir=$TRAIN_DIR \
  --output_dir=$OUTPUT_DIR \
  --resolution=512 \
  --train_batch_size=16 \
  --gradient_accumulation_steps=1 \
  --unet_learning_rate=2e-4 \
  --text_encoder_learning_rate=4e-5 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=100 \
  --max_train_steps=2000 \
  --lora_rank=8 \
  --lora_dropout=0.1 \
  --lora_alpha=16 \
  --resume_from_checkpoint="latest" \
  --report_to tensorboard \
  --validation_steps 100 \
  --checkpointing_steps 100 \
  --mixed_precision="fp16" \
  --use_8bit_adam \
  --set_grads_to_none \
  --enable_xformers_memory_efficient_attention

In [ ]:
!accelerate launch infer.py \
    --model_path=$OUTPUT_DIR \
    --validation_image=$VAL_IMG \
    --validation_mask=$VAL_MASK \
    --output_dir=$OUTPUT_IMG_DIR

In [ ]:
# Zip final inference results
!zip -r9j $OUTPUT_IMG_DIR.zip $OUTPUT_IMG_DIR
# Zip tensorboard logs
!zip -r9D $OUTPUT_DIR-tensorboard.zip $OUTPUT_DIR/logs
%ls